In [4]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import os
mnist = input_data.read_data_sets("/home/zhaocq/桌面/tensorflow/mnist/raw/",one_hot=True)
sess = tf.InteractiveSession()
X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])
#定义权重初始化函数以及权重向量
#定义权重初始化函数以及权重向量
#def init_weights(shapes):
#    return tf.Variable(tf.random_normal(shapes, stddev=0.01))
#w_h = init_weights([784,625])
#w_h2 = init_weights([625,625])
#w_o = init_weights([625,10])
w_h = tf.Variable(tf.random_normal([784,625], stddev=0.01),name = 'w_1')
w_h2 = tf.Variable(tf.random_normal([625,625], stddev=0.01),name = 'w_2')
w_o = tf.Variable(tf.random_normal([625,10], stddev=0.01),name = 'w_0')
variables_dict = {'w_1':w_h, 'w_2':w_h2,  'w_o':w_o}


def model(X, w_h , w_h2 , w_o, p_keep_input,p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))
    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h,w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    return tf.nn.softmax(tf.matmul(h2 ,w_o))
p_keep_input = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
py_x = model(X, w_h,w_h2,w_o,p_keep_input,p_keep_hidden)
#损失函数定义
cost = tf.reduce_mean(-tf.reduce_mean(Y*tf.log(py_x),reduction_indices=[1]))
train_op = tf.train.AdamOptimizer(1e-4).minimize(cost)
predict_op = tf.equal(tf.argmax(py_x,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(predict_op,tf.float32))

ckpt_dir = "./trainsave/ckpt_dir/model.ckpt-997"
global_step = tf.Variable(0,name='global_step',trainable = False)
saver = tf.train.Saver(variables_dict)
non_storable_variable = tf.Variable(777)

Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-labels-idx1-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-labels-idx1-ubyte.gz


/home/zhaocq/anaconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [5]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    saver.restore(sess,ckpt_dir)
    trainaccuracy = sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels,p_keep_hidden:1.0,p_keep_input:1.0})
    print("test accuracy %g"%trainaccuracy)#feed_dict给placeholder创建的tensor赋值

INFO:tensorflow:Restoring parameters from ./trainsave/ckpt_dir/model.ckpt-997
test accuracy 0.9639
